# Closed Cards Report

In [1]:
import time
start_time = time.time()

## 1. Set a time period

In [2]:
start_date = '2023-10-01'
end_date = '2023-10-31'

## 2. Add Output Link

In [3]:
output = "C:/Users/aleksandar.dimitrov/Desktop/PD/Output/ClosedCards.xlsx"

## 3. Wait for the program..


### Importing Libraries

In [4]:
import pandas as pd
import pyodbc
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows

import win32com.client
from datetime import datetime

### Templates Loading

In [5]:
input = "C:/Users/aleksandar.dimitrov/Desktop/PD/Input/Closed_Cards_template/ClosedCards.xlsx"

In [6]:
workbook = load_workbook(input)

In [7]:
worksheet = workbook.active

### SQL Querries Loading

In [8]:
server = "scorpio"
database = "BIsmartWCBG"
conn = pyodbc.connect(f"DRIVER=ODBC Driver 17 for SQL Server;SERVER={server};DATABASE={database};Trusted_Connection=yes")

In [9]:
sql_query = f"""

DECLARE @StartDate DATE = '{start_date}';
DECLARE @EndDate DATE = '{end_date}';

SELECT 
DimOff.ContractNumber AS EasyClientNumber
,CONVERT(DATE, DimOff.DateClosed) AS Date
,DimPr.Name AS Product
,DimCR.Code AS CloseReason    
,COUNT(*) AS Count 
FROM dwh.DimOffers AS DimOff
JOIN dwh.DimOffCloseReason AS DimCR ON DimCR.CloseReasonSK = DimOff.CloseReasonSK
JOIN dwh.DimProduct AS DimPr ON DimPr.ProductSK = DimOff.ProductSK
WHERE DimCR.CloseReasonSK BETWEEN 1 AND 3 
AND CONVERT(DATE, DimOff.DateClosed) BETWEEN @StartDate AND @EndDate
GROUP BY DimOff.ContractNumber, DimPr.Name, DimCR.Code, CONVERT(DATE, DimOff.DateClosed);

"""

In [10]:
data_sql = pd.read_sql_query(sql_query, conn)
data_sql

C:\Users\aleksandar.dimitrov\AppData\Local\Temp\ipykernel_7392\2272456572.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_sql = pd.read_sql_query(sql_query, conn)


,EasyClientNumber,Date,Product,CloseReason,Count
0,400018,2023-10-06,ИАМ,Внесено над лимита,1
1,400056,2023-10-06,ИАМ,Внесено над лимита,1
2,400057,2023-10-06,ИАМ,Внесено над лимита,1
3,400110,2023-10-06,ИАМ,Внесено над лимита,1
4,406455,2023-10-03,ИАМ,Внесено над лимита,1
...,...,...,...,...,...
2160,11061752,2023-10-23,Бяла Карта 3,Внесено над лимита,1
2161,11062110,2023-10-06,A1 - 8.5%,Внесено над лимита,1
2162,11062325,2023-10-10,Бяла Карта 3,Внесено над лимита,1
2163,11063071,2023-10-18,Бяла Карта Gold - 10%,Внесено над лимита,1


### Data Generation

In [11]:
data_sql["CloseReason"] = data_sql["CloseReason"].replace({
    "Внесено над лимита": "VoluntaryChurn", 
    "Недостиг до зануляване": "VoluntaryChurn",
    "Цесия": 'Cession'
})

data_sql["Date"] = pd.to_datetime(data_sql["Date"]).dt.strftime("%d.%m.%Y")
data_sql

,EasyClientNumber,Date,Product,CloseReason,Count
0,400018,06.10.2023,ИАМ,VoluntaryChurn,1
1,400056,06.10.2023,ИАМ,VoluntaryChurn,1
2,400057,06.10.2023,ИАМ,VoluntaryChurn,1
3,400110,06.10.2023,ИАМ,VoluntaryChurn,1
4,406455,03.10.2023,ИАМ,VoluntaryChurn,1
...,...,...,...,...,...
2160,11061752,23.10.2023,Бяла Карта 3,VoluntaryChurn,1
2161,11062110,06.10.2023,A1 - 8.5%,VoluntaryChurn,1
2162,11062325,10.10.2023,Бяла Карта 3,VoluntaryChurn,1
2163,11063071,18.10.2023,Бяла Карта Gold - 10%,VoluntaryChurn,1


### Data Export

In [12]:
data_rows = list(dataframe_to_rows(data_sql, index=False, header=False))

In [13]:
start_row = 2 
start_column = 1 

In [14]:
for row in data_rows:
    for value in row:
        worksheet.cell(row=start_row, column=start_column, value=value)
        start_column += 1
    start_column = 1
    start_row += 1; 

In [15]:
workbook.save(output)

### Email Anouncing

In [16]:
recipient_email = "alexi.zein@gmail.com"
subject = "Closed Cards"
message = """Dear colleagues,

This email is automatically generated and contains information about added data.

Date and time of generation: {}

The Closed Cards data has been added successfully and can be found in the shared folder on PD.

""".format(datetime.now().strftime("%d %B %Y, %H:%M ч."))

In [17]:
outlook = win32com.client.Dispatch("Outlook.Application")
namespace = outlook.GetNamespace("MAPI")

In [18]:
mail = outlook.CreateItem(0)
mail.Subject = subject
mail.Body = message
mail.To = recipient_email

In [19]:
mail.Send()

In [20]:
print("The notification email has been successfully sent.")

The notification email has been successfully sent.


In [21]:
end_time = time.time()
execution_time_seconds = end_time - start_time
execution_time_minutes = execution_time_seconds / 60
print(f"The script executed for {execution_time_minutes:.2f} minutes.")

The script executed for 0.03 minutes.
